<img src=https://audiovisuales.icesi.edu.co/assets/custom/images/ICESI_logo_prin_descriptor_RGB_POSITIVO_0924.jpg width=200>

*Milton Orlando Sarria Paja, PhD.*

----
Es esta guía de trabajo se abordan **tres conjuntos de datos** con características y retos distintos, lo que permite explorar la aplicación de modelos estadísticos y de aprendizaje automático en contextos variados. El enfoque principal será evaluar la capacidad de los **modelos lineales**, junto con técnicas de **regularización** y **validación cruzada**, para identificar variables relevantes y seleccionar el mejor modelo en cada caso.

Cada dataset representa un desafío específico:

* El **dataset de propinas (Tips)** se trabajará como un problema de **regresión**, en el que la variable objetivo es el **monto total de la cuenta (`total_bill`)**. El interés se centra en analizar qué factores (propinas, número de comensales, día de la semana, entre otros) explican mejor el valor final de la cuenta.
* El **dataset de cáncer de mama (Breast Cancer)** constituye un problema de **clasificación médica**, cuyo reto es distinguir entre tumores **malignos** y **benignos** a partir de mediciones celulares, destacando la importancia de identificar variables clave para la detección temprana.
* El **dataset del Titanic (Seaborn)** se plantea como un caso de **clasificación histórica y social**, donde el objetivo es predecir la **supervivencia de los pasajeros** en función de variables demográficas y de contexto. Este dataset enfrenta retos como valores faltantes y variables categóricas.


El **objetivo general** es explorar la **potencialidad de los modelos lineales**, evaluando su capacidad tanto en **problemas de regresión** como de **clasificación**. Para mejorar la robustez y la interpretación:

* Se emplearán técnicas de **regularización** (Ridge, Lasso), que permiten controlar el sobreajuste y seleccionar variables relevantes.
* Se aplicará la **validación cruzada** como estrategia de comparación para determinar el mejor modelo en cada caso.

De esta manera, se busca no solo construir modelos predictivos, sino también **comprender cómo influyen distintas variables en fenómenos de diversa naturaleza**: consumo en restaurantes, diagnóstico médico y supervivencia en un evento histórico.

----

Preparar entorno


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

----


## CASO 1

**Tips Dataset** (dataset de propinas) 

Este dataset proviene de observaciones en un restaurante y contiene información sobre las cuentas de clientes y las propinas que dieron.

### 📊 Descripción del dataset **Tips**

* **Origen**: Fue creado por el estadístico Brian W. Kernighan y popularizado en el libro *“The Elements of Statistical Learning”* y en librerías como **Seaborn** en Python.
* **Número de filas**: 244
* **Número de columnas**: 7

### 📑 Variables del dataset

1. **total\_bill** (*float*) → Importe total de la cuenta en dólares.
2. **tip** (*float*) → Propina entregada en dólares.
3. **sex** (*string*) → Sexo de la persona que pagó la cuenta (`Male` / `Female`).
4. **smoker** (*string*) → Indica si en la mesa había fumadores (`Yes` / `No`).
5. **day** (*string*) → Día de la semana en que ocurrió el consumo (`Thur`, `Fri`, `Sat`, `Sun`).
6. **time** (*string*) → Momento del día (`Lunch` / `Dinner`).
7. **size** (*int*) → Número de personas en la mesa.

### Tarea

* Analizar la relación entre el **total de la cuenta y la propina**.
* Estudiar diferencias de propina según el **sexo**, si había **fumadores**, el **día** y la **hora**.
* Ejemplos de **visualización de datos** con Seaborn/Matplotlib (gráficos de dispersión, boxplots, regresiones).
* Casos de enseñanza en **regresión lineal, ANOVA, y estadística descriptiva**.


In [2]:
# cargar los datos desde seaborn
tips = sns.load_dataset('tips')
tips.head(3)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3


#### Ejercicio 1
 1. Separar X, de y
 2. Crear un pipeline: Convertir variables categoricas en numericas con one-hot-encoding, normalizar variables numericas con standardScaler

In [4]:
# Define features and target variable
X = tips.drop('total_bill', axis=1)
y = tips['total_bill']

# Identificar variables numéricas y categóricas


['tip', 'size']
['sex', 'smoker', 'day', 'time']


In [5]:
# Preprocesamiento


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 ['tip', 'size']),
                                ('cat',
                                 Pipeline(steps=[('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['sex', 'smoker', 'day', 'time'])])

## Actividad: Validación Cruzada con Regresión Lineal

En esta actividad pondrás en práctica la técnica de **validación cruzada (K-Fold Cross-Validation)** para evaluar el desempeño de un modelo de regresión lineal. La validación cruzada es una estrategia que permite dividir los datos en varias particiones (o *folds*), entrenar el modelo en una parte y evaluarlo en la otra, repitiendo el proceso varias veces para obtener una medida más robusta del error del modelo.

### Instrucciones

1. **Configura la validación cruzada** usando `KFold` con 5 particiones (*n\_splits=5*), habilitando `shuffle=True` para mezclar los datos y fijando `random_state=42` para asegurar reproducibilidad.

2. **Inicializa un modelo de regresión lineal** con `LinearRegression()` de *scikit-learn*.

3. **Divide los datos en entrenamiento y validación** para cada fold que genere `KFold`.

   * Usa los índices de `train_index` y `val_index` para separar `X_train, X_val` y `y_train, y_val`.

4. **Entrena el modelo**  en los datos de entrenamiento de cada fold.

5. **Genera predicciones** sobre los datos de validación.

6. **Calcula el error** en cada fold usando la métrica **Error Cuadrático Medio (MSE)** con `mean_squared_error`.

7. **Imprime los resultados** mostrando el número de fold y el MSE correspondiente.

### 📌 Resultado esperado

Al finalizar, deberías ver en consola algo como:

```
Fold 1 - MSE: ...
Fold 2 - MSE: ...
Fold 3 - MSE: ...
Fold 4 - MSE: ...
Fold 5 - MSE: ...
```



In [7]:

# Validación cruzada - Regresión Lineal



Regresión Lineal - Validación Cruzada
Fold 1 - MSE: 31.873635550729944
Fold 2 - MSE: 27.265990214952172
Fold 3 - MSE: 38.018476635453496
Fold 4 - MSE: 39.137556904183214
Fold 5 - MSE: 42.392393576414825

MSE promedio: 35.73761057634673

MSE desv. std: 5.435931867415825


#### Regularización
Aplicar modelos de regresión lineal con regularización (Ridge y Lasso) al dataset Tips para predecir la variable **total_bill**, identificando qué variables explicativas son más relevantes y comparando los efectos de la regularización sobre los coeficientes.

1. Seleccionar el mejor alpha usando CV
2. Validar de nuevo el rendimiento del regresor usando el modelo ajustado
3. Hacer el proceso para Lasso y Ridge

In [9]:
# Lasso Regression (selección de variables)

# Evaluar rendimiento



Mejor alpha (Lasso): 0.1386845609390539
Regresión Lasso - Validación Cruzada
Fold 1 - MSE: 31.272816972635642
Fold 2 - MSE: 30.691960445153416
Fold 3 - MSE: 38.29636409287762
Fold 4 - MSE: 40.42021328808717
Fold 5 - MSE: 41.906158484456945

Lasso - MSE promedio: 36.51750265664215
Lasso - MSE std: 4.6664071810038585


In [11]:
# Ridge Regression (comparación)
#

# Evaluar rendimiento



Mejor alpha (Ridge): 14.563484775012444
Regresión Ridge - Validación Cruzada
Fold 1 - MSE: 31.942249580857073
Fold 2 - MSE: 26.89929711583635
Fold 3 - MSE: 37.99233426100507
Fold 4 - MSE: 39.894708801241954
Fold 5 - MSE: 41.353998424221324

Ridge - MSE promedio: 35.61651763663235

Ridge - MSE std: 5.410291848235538


# Caso 2
---

**Dataset de Cáncer de Mama (Breast Cancer Dataset)**

El **cáncer de mama** es el tipo de cáncer más común entre las mujeres en el mundo. Representa aproximadamente **25% de todos los casos de cáncer** y, en 2015, afectó a más de **2.1 millones de personas**.
Este dataset es ampliamente utilizado en **clasificación supervisada**, con el objetivo principal de **predecir si un tumor es maligno (canceroso) o benigno (no canceroso)**, a partir de características obtenidas de imágenes de aspiraciones con aguja fina (FNA) del tejido mamario.

### 📑 Características generales del dataset

* **Origen**: Wisconsin Diagnostic Breast Cancer (WDBC), creado por Dr. William H. Wolberg (Universidad de Wisconsin).
* **Número de muestras**: 569 pacientes.
* **Número de variables (features)**: 30 características numéricas derivadas de imágenes de tejido.
* **Variable objetivo (target)**: 2 clases

  * `Malignant` (M) → Tumor maligno.
  * `Benign` (B) → Tumor benigno.

---

## Variables del dataset

Las características se calculan a partir de imágenes digitalizadas de masas mamarias. Cada variable describe propiedades de los núcleos celulares.

Se dividen en **10 medidas básicas**, cada una evaluada en **3 versiones**:

* **mean** (promedio),
* **se** (error estándar),
* **worst** (valor máximo o más grande).

### 🔬 Lista de variables (30 features)

1. **radius** (radio medio de los núcleos celulares)
2. **texture** (desviación estándar de los valores de grises)
3. **perimeter** (perímetro)
4. **area** (área)
5. **smoothness** (variación local en la longitud de los radios)
6. **compactness** (perímetro² / área - 1.0)
7. **concavity** (severidad de las concavidades)
8. **concave points** (número de puntos cóncavos del contorno)
9. **symmetry** (simetría)
10. **fractal dimension** (aproximación de la “complejidad” del borde)

Y cada una con sus 3 medidas:

* `mean` (ej. radius\_mean, texture\_mean, …),
* `se` (ej. radius\_se, texture\_se, …),
* `worst` (ej. radius\_worst, texture\_worst, …).

En total: **30 columnas de variables predictoras**.

---

## Tarea 

* Clasificación binaria (Maligno vs. Benigno).
  

In [13]:
np.random.seed(42)
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df.head(3)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.0,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0


## Ejercicio 1

Hacer un experimento inicial para establecer el baseline:
1. Separar X, de y
2. Crear cojuntos de entrnamiento y prueba : 70%-30%. Usar la siguiente configuración: train_test_split(x, y, test_size=0.30, stratify=y, random_state=42)
3. Usar normalizador: StandardScaler
4. Entrenar un modelo de regresión logistica sin ninguna penalidad, evaluar rendimiento


In [55]:
#1

#2


In [ ]:
#3

#4
log_model = LogisticRegression(penalty=None)
#  Ajustar el modelo con el conjunto de entrenamiento normalizado



Dimensiones del conjunto de entrenamiento: (398, 30)
Dimensiones del conjunto de prueba: (171, 30)


Intercepto: [-44.29078732]
                         Coeficiente
mean radius                37.359575
mean texture               14.855189
mean perimeter             37.567281
mean area                   9.890719
mean smoothness            -6.145943
mean compactness          240.834519
mean concavity            -74.577640
mean concave points       -72.046986
mean symmetry               4.409757
mean fractal dimension   -141.748019
radius error             -140.074499
texture error              49.517271
perimeter error            57.394840
area error               -184.333470
smoothness error           -4.027969
compactness error         -76.984142
concavity error           133.600914
concave points error      -56.447846
symmetry error             78.422902
fractal dimension error    30.069437
worst radius              -68.211928
worst texture            -114.503951
worst perimeter           -20.768434
worst area               -114.599393
worst smoothness           -1.152200
worst compa

Matriz de Confusión:
[[ 59   5]
 [  1 106]]

Accuracy: 0.9649


### Ejercicio 2

Usar Validación cruzada estratificada para corroborar los resultados anteriores. Imprimir el rendimiento en cada fold. Calcular el promedio

In [66]:
# 1. Cargar dataset


# 2. Validación cruzada estratificada
# Definimos pipeline: escalado + regresión logística
log_reg = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=5000, penalty=None))  # sin regularización
])

#3 Definir validación cruzada estratificada

#4 Evaluar con accuracy

print("Validación cruzada - Regresión Logística (sin regularización)")


Validación cruzada - Regresión Logística (sin regularización)
Accuracy por fold: [0.92982456 0.95614035 0.92982456 0.97368421 0.97345133]
Accuracy promedio: 0.952585002328831


### Ejercicio 3
1. Aplicar Lasso para identificar variables relevantes. Usar C=1
2. Reentrenar y validar de nuevo usando solamente las variables seleccionadas

In [67]:
#  Aplicar L1 (Lasso) para selección de variables

# Modelo con regularización L1 (Lasso)
lasso_model = LogisticRegression(
    penalty="l1", solver="saga", max_iter=5000, C=1.0
)

# Escalamos antes de entrenar

# Entrenamos con todas las variables

print("\nVariables seleccionadas por Lasso:")



Variables seleccionadas por Lasso:
['mean concavity', 'mean concave points', 'mean fractal dimension', 'radius error', 'texture error', 'smoothness error', 'compactness error', 'fractal dimension error', 'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 'worst concavity', 'worst concave points', 'worst symmetry']


In [68]:
coef

mean radius                0.000000
mean texture               0.000000
mean perimeter             0.000000
mean area                  0.000000
mean smoothness            0.000000
mean compactness           0.000000
mean concavity            -0.103455
mean concave points       -1.071049
mean symmetry              0.000000
mean fractal dimension     0.146945
radius error              -2.697941
texture error              0.377217
perimeter error            0.000000
area error                 0.000000
smoothness error          -0.324727
compactness error          0.896574
concavity error            0.000000
concave points error       0.000000
symmetry error             0.000000
fractal dimension error    0.221898
worst radius              -1.975675
worst texture             -1.770723
worst perimeter           -0.594891
worst area                -1.805904
worst smoothness          -0.541172
worst compactness          0.000000
worst concavity           -1.129958
worst concave points      -1

In [70]:
#  Reentrenar usando solo variables seleccionadas

X_selected = X[selected_features]

log_reg_selected = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=5000, penalty=None))
])

scores_selected = cross_val_score(log_reg_selected, X_selected, y, cv=cv, scoring="accuracy")

print("\nValidación cruzada - Modelo con variables seleccionadas por Lasso")
print("Accuracy por fold:", scores_selected)
print("Accuracy promedio:", np.mean(scores_selected))


Validación cruzada - Modelo con variables seleccionadas por Lasso
Accuracy por fold: [0.96491228 0.94736842 0.92105263 1.         0.99115044]
Accuracy promedio: 0.9648967551622418


# Caso 3

🚢 **Titanic Dataset** 

El hundimiento del **RMS Titanic** ocurrió el **15 de abril de 1912**, en su viaje inaugural entre Southampton (Inglaterra) y Nueva York (EE. UU.), tras chocar con un iceberg. A bordo viajaban más de **2200 pasajeros y tripulantes**, y más de **1500 personas murieron**, lo que lo convierte en uno de los desastres marítimos más trágicos de la historia.

El dataset fue popularizado por **Kaggle** en la competencia *"Titanic: Machine Learning from Disaster"*. El objetivo principal es **predecir la probabilidad de supervivencia de los pasajeros** a partir de la información disponible.

Seaborn incluye una versión preprocesada con algunas variables renombradas, transformadas o creadas para facilitar análisis exploratorio y visualización

---

## 📑 Características generales del dataset

* **Número de registros**:

  * 891 pasajeros.

* **Variable objetivo (target)**:

  * `Survived` → indica si un pasajero sobrevivió (`1`) o no (`0`).
---
## Variables del dataset Titanic

* **survived** (*int, 0/1*) → Indica si el pasajero sobrevivió (`1`) o no (`0`).
* **pclass** (*int, 1/2/3*) → Clase del pasajero (1ª = alta, 2ª = media, 3ª = baja).
* **sex** (*string*) → Sexo del pasajero (`male`, `female`).
* **age** (*float*) → Edad del pasajero en años. Puede tener valores faltantes.
* **sibsp** (*int*) → Número de hermanos/esposos a bordo.
* **parch** (*int*) → Número de padres/hijos a bordo.
* **fare** (*float*) → Tarifa del boleto en libras.
* **embarked** (*string*) → Puerto de embarque (`C` = Cherbourg, `Q` = Queenstown, `S` = Southampton).
* **class** (*string*) → Versión categórica de `pclass` (`First`, `Second`, `Third`).
* **who** (*string*) → Tipo de pasajero según sexo y edad (`man`, `woman`, `child`).
* **adult\_male** (*bool*) → Verdadero si el pasajero era un hombre adulto (edad >16 y sexo masculino).
* **deck** (*string*) → Cubierta del barco (derivada de la cabina, valores `A`–`G`, muchos faltantes).
* **embark\_town** (*string*) → Nombre del puerto de embarque (`Cherbourg`, `Queenstown`, `Southampton`).
* **alive** (*string*) → Estado de supervivencia (`yes`, `no`), equivalente a `survived`.
* **alone** (*bool*) → Verdadero si el pasajero viajaba solo (`sibsp + parch = 0`).


---

## Tarea

* Clasificación binaria (`Survived` = 0 o 1).
* Preprocesamiento y manejo de **valores faltantes y variables categóricas**.
* Feature engineering (ejemplo: agrupar tamaño de familias con `SibSp` + `Parch`).
* Comparar modelos de ML 

## Variables que **no generan leakage**, pero pueden ser redundantes:

* `survived` → es el **target**, nunca debe estar como feature.
* `alive` → es simplemente una versión categórica de `survived` → **hay que eliminarla** porque equivale al target.
* `class` → es redundante con `pclass`. Puedes quedarte con una de las dos (según prefieras numérica o categórica).
* `embark_town` → es redundante con `embarked`. 

---

## ⚠️ Variables con **muchos valores faltantes**:

* `deck` → tiene muchísimos NaN. Puedes decidir eliminarla o imputar valores, pero si no la procesas bien puede dañar el modelo.

---

## ✅ Variables útiles para predicción:

* `pclass`, `sex`, `age`, `sibsp`, `parch`, `fare`, `embarked`
* Variables derivadas (`who`, `adult_male`, `alone`) pueden ser útiles, pero ya están correlacionadas con las anteriores.

---

## 🎯 Recomendación práctica

Para evitar problemas de leakage, multicolinealidad o redundancia:

1. **Eliminar:**

   * `alive` (porque es igual al target).
   * Una de entre (`pclass`, `class`).
   * Una de entre (`embarked`, `embark_town`).
2. **Revisar:**

   * `deck` → si decides mantenerla, tendrás que imputar muchos valores.
3. **Mantener:**

   * `sex`, `age`, `sibsp`, `parch`, `fare`, `alone` (y/o combinaciones como `family_size = sibsp + parch + 1`).


In [14]:
# Cargar datos
titanic_df = sns.load_dataset('titanic')
titanic_df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


In [ ]:
# Selección de variables útiles


In [76]:
# Identificar variables numéricas y categóricas


# Preprocesamiento



ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['age', 'fare', 'sibsp', 'parch']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['pclass', 'sex', 'embarked', 'deck', 'alone',
                                  'who', 'adult_male'])])

In [80]:
# ===============================
# Validación cruzada con Regresión Logística (sin regularización)
# ===============================



Validación cruzada - Regresión Logística (sin regularización)
Accuracy por fold: [0.83798883 0.83146067 0.78651685 0.80898876 0.84269663]
Accuracy promedio: 0.8215303496327915


In [81]:
# ===============================
# Aplicar LASSO (L1) para selección de variables
# ===============================


# Extraer coeficientes después de one-hot




Variables seleccionadas por Lasso:
deck_E             0.634755
deck_D             0.245786
fare               0.197331
who_woman          0.075455
deck_G            -0.034627
sex_male          -0.104309
age               -0.258247
parch             -0.274709
embarked_S        -0.352465
pclass_2          -0.370420
alone_True        -0.469078
sibsp             -0.618740
deck_C            -0.655840
who_man           -1.399936
adult_male_True   -1.399936
pclass_3          -1.420543
dtype: float64


In [82]:
coef

age               -0.258247
fare               0.197331
sibsp             -0.618740
parch             -0.274709
pclass_2          -0.370420
pclass_3          -1.420543
sex_male          -0.104309
embarked_Q         0.000000
embarked_S        -0.352465
deck_B             0.000000
deck_C            -0.655840
deck_D             0.245786
deck_E             0.634755
deck_F             0.000000
deck_G            -0.034627
alone_True        -0.469078
who_man           -1.399936
who_woman          0.075455
adult_male_True   -1.399936
dtype: float64

In [83]:

# Reentrenar con validación cruzada usando Lasso





Validación cruzada - Modelo con Lasso (variables seleccionadas)
Accuracy por fold: [0.83798883 0.83146067 0.79775281 0.82022472 0.85393258]
Accuracy promedio: 0.8282719226664993
